In [1]:
# enable autoreload
%load_ext autoreload
%autoreload 2

In [2]:
from askharrison.arxiv_search import expand_arxiv_query, run_multi_arixv_queries

In [3]:
problem_statement = """RAG(retrieval augmented generation) sometimes the sources retrieved are relevant but not enough to answer the question user ask. How do research approach this case"""

In [23]:
problem_statement = """reduce hallucination in RAG(retrieval augmented generation), make queries tailored for arxiv search, and then use the retrieved papers to answer the question"""

In [24]:
search_queries = expand_arxiv_query(problem_statement)

In [25]:
search_queries

['Reducing hallucination in retrieval augmented generation models',
 'Techniques for minimizing hallucination in RAG models',
 'Enhancing the effectiveness of Retrieval Augmented Generation',
 'Studies on improving the accuracy of Retrieval Augmented Generation',
 'Methods to reduce artifact generation in RAG models',
 'Optimization techniques in retrieval augmented generation models',
 'Novel methods for reducing hallucination in RAG',
 'Approaches for improving reliability in retrieval augmented generation models',
 'Research on limiting hallucination in RAG',
 'Advanced strategies for retrieval augmented generation optimization']

In [26]:
arxiv_query_results = run_multi_arixv_queries(search_queries)

100%|██████████| 10/10 [00:25<00:00,  2.53s/it]


In [27]:
len(arxiv_query_results.items())

10

In [28]:
# flatten arxiv_query_results
all_results = []
for query in arxiv_query_results:
    for result in arxiv_query_results[query]:
        all_results.append(result)

# make arxiv query results a dataframe and create a new dataframe with only unique entry_id
import pandas as pd

arixv_result_df = pd.DataFrame(all_results)
unique_arixv_result_df = arixv_result_df.drop_duplicates(subset='entry_id')

In [29]:
arixv_result_df.shape, unique_arixv_result_df.shape

((200, 13), (159, 13))

In [30]:
arixv_result_df.groupby('entry_id').count().sort_values('title', ascending=False).head(10)

,authors,categories,comment,doi,journal_ref,links,pdf_url,primary_category,published,summary,title,updated
entry_id,,,,,,,,,,,,
http://arxiv.org/abs/2410.18251v1,4,4,4,0,0,4,4,4,4,4,4,4
http://arxiv.org/abs/2410.22353v1,3,3,0,0,0,3,3,3,3,3,3,3
http://arxiv.org/abs/2401.15391v1,3,3,3,0,0,3,3,3,3,3,3,3
http://arxiv.org/abs/2410.17783v1,3,3,3,0,0,3,3,3,3,3,3,3
http://arxiv.org/abs/2401.05856v1,3,3,0,0,0,3,3,3,3,3,3,3
http://arxiv.org/abs/2410.12248v1,3,3,0,0,0,3,3,3,3,3,3,3
http://arxiv.org/abs/2406.19150v1,3,3,0,0,0,3,3,3,3,3,3,3
http://arxiv.org/abs/2408.15533v2,3,3,0,0,0,3,3,3,3,3,3,3
http://arxiv.org/abs/2410.11414v2,3,3,3,0,0,3,3,3,3,3,3,3


In [31]:
from askharrison.prompts.content_curation import create_arxiv_filtering_prompt

In [32]:
help(create_arxiv_filtering_prompt)

Help on function create_arxiv_filtering_prompt in module askharrison.prompts.content_curation:

create_arxiv_filtering_prompt(problem_statement: str, doc_abstract: str)



In [33]:
# create a prompt for each arxiv entry
arxiv_reranking_prompts = [create_arxiv_filtering_prompt(problem_statement, 
                                         record['title']+"\n"+record['summary']) for record in unique_arixv_result_df.to_dict(orient='records')]

In [34]:
from askharrison.llm_models import parallel_llm_processor, process_question, safe_eval, extract_python_code

In [35]:
reranking_llm_response = parallel_llm_processor(arxiv_reranking_prompts, llm_function=process_question, 
                                                max_workers=8)

Processing prompts: 100%|██████████| 159/159 [01:11<00:00,  2.22it/s]


In [36]:
llm_responses_results = [safe_eval(extract_python_code(response)) for response in reranking_llm_response]
# filter out empty responses

unique_arixv_result_df.shape, len(llm_responses_results)

((159, 13), 159)

In [37]:
# extract reasoning, is_direct, 'is_relevant' from llm_responses_results if it is not empty, and add to unique_arixv_result_df
unique_arixv_result_df['reasoning'] = [response['reasoning'] if response else None for response in llm_responses_results]
unique_arixv_result_df['is_direct'] = [response['is_direct'] if response else None for response in llm_responses_results]
unique_arixv_result_df['is_relevant'] = [response['is_relevant'] if response else None for response in llm_responses_results]


C:\Users\alist\AppData\Local\Temp\ipykernel_103088\658567496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_arixv_result_df['reasoning'] = [response['reasoning'] if response else None for response in llm_responses_results]
C:\Users\alist\AppData\Local\Temp\ipykernel_103088\658567496.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_arixv_result_df['is_direct'] = [response['is_direct'] if response else None for response in llm_responses_results]
C:\Users\alist\AppData\Local\Temp\ipykerne

In [38]:
# increase max column width in pandas
pd.set_option('display.max_colwidth', None)

In [39]:
unique_arixv_result_df.query('is_direct == True').shape

(15, 16)

In [40]:
unique_arixv_result_df.query('is_direct == True')\
    [['title','entry_id', 'summary', 'is_relevant']]

,title,entry_id,summary,is_relevant
6,Context-Augmented Code Generation Using Programming Knowledge Graphs,http://arxiv.org/abs/2410.18251v1,"Large Language Models (LLMs) and Code-LLMs (CLLMs) have significantly\nimproved code generation, but, they frequently face difficulties when dealing\nwith challenging and complex problems. Retrieval-Augmented Generation (RAG)\naddresses this issue by retrieving and integrating external knowledge at the\ninference time. However, retrieval models often fail to find most relevant\ncontext, and generation models, with limited context capacity, can hallucinate\nwhen given irrelevant data. We present a novel framework that leverages a\nProgramming Knowledge Graph (PKG) to semantically represent and retrieve code.\nThis approach enables fine-grained code retrieval by focusing on the most\nrelevant segments while reducing irrelevant context through a tree-pruning\ntechnique. PKG is coupled with a re-ranking mechanism to reduce even more\nhallucinations by selectively integrating non-RAG solutions. We propose two\nretrieval approaches-block-wise and function-wise-based on the PKG, optimizing\ncontext granularity. Evaluations on the HumanEval and MBPP benchmarks show our\nmethod improves pass@1 accuracy by up to 20%, and outperforms state-of-the-art\nmodels by up to 34% on MBPP. Our contributions include PKG-based retrieval,\ntree pruning to enhance retrieval precision, a re-ranking method for robust\nsolution selection and a Fill-in-the-Middle (FIM) enhancer module for automatic\ncode augmentation with relevant comments and docstrings.",4
10,Coarse-to-Fine Highlighting: Reducing Knowledge Hallucination in Large Language Models,http://arxiv.org/abs/2410.15116v1,"Generation of plausible but incorrect factual information, often termed\nhallucination, has attracted significant research interest. Retrieval-augmented\nlanguage model (RALM) -- which enhances models with up-to-date knowledge --\nemerges as a promising method to reduce hallucination. However, existing RALMs\nmay instead exacerbate hallucination when retrieving lengthy contexts. To\naddress this challenge, we propose COFT, a novel\n\textbf{CO}arse-to-\textbf{F}ine highligh\textbf{T}ing method to focus on\ndifferent granularity-level key texts, thereby avoiding getting lost in lengthy\ncontexts. Specifically, COFT consists of three components: \textit{recaller},\n\textit{scorer}, and \textit{selector}. First, \textit{recaller} applies a\nknowledge graph to extract potential key entities in a given context. Second,\n\textit{scorer} measures the importance of each entity by calculating its\ncontextual weight. Finally, \textit{selector} selects high contextual weight\nentities with a dynamic threshold algorithm and highlights the corresponding\nparagraphs, sentences, or words in a coarse-to-fine manner. Extensive\nexperiments on the knowledge hallucination benchmark demonstrate the\neffectiveness of COFT, leading to a superior performance over $30\%$ in the F1\nscore metric. Moreover, COFT also exhibits remarkable versatility across\nvarious long-form tasks, such as reading comprehension and question answering.",4
14,The Effects of Hallucinations in Synthetic Training Data for Relation Extraction,http://arxiv.org/abs/2410.08393v1,"Relation extraction is crucial for constructing knowledge graphs, with large\nhigh-quality datasets serving as the foundation for training, fine-tuning, and\nevaluating models. Generative data augmentation (GDA) is a common approach to\nexpand such datasets. However, this approach often introduces hallucinations,\nsuch as spurious facts, whose impact on relation extraction remains\nunderexplored. In this paper, we examine the effects of hallucinations on the\nperformance of relation extraction on the document and sentence levels. Our\nempirical study reveals that hallucinations considerably compromise the ability\nof models to extract relations from text, with recall reductions between 19.1%\nand 39.2%. We identify t